<a href="https://colab.research.google.com/drive/1gxDRxQ2mxGcrorKW_Vuj87AiupuCh-uU?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>

#	🔄 What is Self-Adaptive RAG?

Self-Adaptive RAG is an advanced technique that autonomously optimizes its performance over time. It uses machine learning algorithms to continuously analyse its own outputs, user feedback, and performance metrics to refine its retrieval and generation strategies. This system can adjust its parameters, update its knowledge base, and modify its decision-making processes without constant human intervention, allowing it to adapt to changing information landscapes and user needs.

# 🔧 Self-Adaptive RAG Implementation:

1. **Query Complexity Assessment:** We use the LLM to rate the complexity of the query on a scale of 1 to 5.

2. **Adaptive Retrieval Strategy:**

  * For simple queries (complexity <= 2), we use standard retrieval.
  * For moderately complex queries (complexity <= 4), we use query expansion before retrieval.
  * For very complex queries (complexity > 4), we use hypothetical document embedding (HyDE) for retrieval.

3. **Final Response Generation:** Using the retrieved context, we generate a final response to the original query.

# ⚙️ Setup

1. **[LLM](https://deepmind.google/technologies/gemini/pro/):** Google's free gemini-pro api endpoint ([Google's API Key](https://console.cloud.google.com/apis/credentials))
2. **[Vector Store](https://www.pinecone.io/learn/vector-database/):** [ChromaDB](https://www.trychroma.com/)
3. **[Embedding Model](https://qdrant.tech/articles/what-are-embeddings/):** [nomic-embed-text-v1.5](https://www.nomic.ai/blog/posts/nomic-embed-text-v1)
4. **[LLM Framework](https://python.langchain.com/v0.2/docs/introduction/):** LangChain
5. **[Huggingface API Key](https://huggingface.co/settings/tokens)**


# Install required libraries

In [ ]:
!pip install -q -U \
     Sentence-transformers==3.0.1 \
     langchain==0.2.11 \
     langchain-google-genai==1.0.7 \
     langchain-chroma==0.1.2 \
     langchain-community==0.2.10 \
     langchain-huggingface==0.0.3 \
     einops==0.8.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.9/379.9 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 12.1 MB/s et

# Import related libraries related to Langchain, HuggingfaceEmbedding

In [ ]:
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain.document_loaders import WebBaseLoader

In [ ]:
import os
import getpass

# Provide Google API Key. You can create Google API key at following link

[Google Gemini-Pro API Creation Link](https://console.cloud.google.com/apis/credentials)

[YouTube Video](https://www.youtube.com/watch?v=ZHX7zxvDfoc)



In [ ]:
os.environ["GOOGLE_API_KEY"] = getpass.getpass()

··········


# Provide Huggingface API Key. You can create Huggingface API key at following link

[Huggingface API Creation Link](https://huggingface.co/settings/tokens)




In [ ]:
os.environ["HF_TOKEN"] = getpass.getpass()

··········


# Step 1: Load and preprocess data code

In [ ]:
def load_and_process_data(url):
    # Load data from web
    loader = WebBaseLoader(url)
    data = loader.load()

    # Split text into chunks (Experiment with Chunk Size and Chunk Overlap to get optimal chunking)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    chunks = text_splitter.split_documents(data)

    return chunks

# Step 2: Create vector store code

In [ ]:
def create_vector_store(chunks):
    embeddings = HuggingFaceEmbeddings(model_name="nomic-ai/nomic-embed-text-v1.5", model_kwargs = {'trust_remote_code': True})
    vectorstore = Chroma.from_documents(chunks, embeddings)
    return vectorstore

# Step 3: Self-Adaptive RAG related code

1. **Query Complexity Assessment:** We use the LLM to rate the complexity of the query on a scale of 1 to 5.

2. **Adaptive Retrieval Strategy:**

  * For simple queries (complexity <= 2), we use standard retrieval.
  * For moderately complex queries (complexity <= 4), we use query expansion before retrieval.
  * For very complex queries (complexity > 4), we use hypothetical document embedding (HyDE) for retrieval.

3. **Final Response Generation:** Using the retrieved context, we generate a final response to the original query.

In [ ]:
def self_adaptive_rag(query, vectorstore, llm):
    # Assess query complexity and type
    assess_prompt = ChatPromptTemplate.from_template(
        "Analyze the following query and provide:\n"
        "1. Complexity (rate from 1-5, where 1 is very simple and 5 is very complex)\n"
        "2. Query type (e.g., factual, analytical, open-ended)\n"
        "3. Suggested retrieval strategy (e.g., standard, query expansion, multi-hop)\n"
        "Query: {query}\n"
        "Analysis:"
    )
    assess_chain = assess_prompt | llm
    try:
        assessment = assess_chain.invoke({"query": query})
        # Parse the assessment (This is a simplified parsing, you might want to make it more robust)
        lines = assessment.content.split('\n')
        complexity = int(lines[0].split(':')[-1].strip())
        query_type = lines[1].split(':')[-1].strip()
        retrieval_strategy = lines[2].split(':')[-1].strip()
    except Exception as e:
        print(f"Error assessing query: {e}")
        complexity, query_type, retrieval_strategy = 3, "unknown", "standard"

    # Adapt retrieval strategy based on assessment
    if retrieval_strategy == "standard" or complexity <= 2:
        docs = vectorstore.similarity_search(query, k=3)
    elif retrieval_strategy == "query expansion" or complexity == 3:
        expand_prompt = ChatPromptTemplate.from_template(
            "Expand the following query with relevant keywords:\nQuery: {query}\nExpanded query:"
        )
        expand_chain = expand_prompt | llm
        expanded_query = expand_chain.invoke({"query": query}).content
        docs = vectorstore.similarity_search(expanded_query, k=4)
    else:  # multi-hop or high complexity
        hop1_prompt = ChatPromptTemplate.from_template(
            "What intermediate question should be answered first to help address this query?\nQuery: {query}\nIntermediate question:"
        )
        hop1_chain = hop1_prompt | llm
        intermediate_query = hop1_chain.invoke({"query": query}).content
        intermediate_docs = vectorstore.similarity_search(intermediate_query, k=2)
        docs = vectorstore.similarity_search(query, k=3)
        docs.extend(intermediate_docs)

    context = "\n\n".join([doc.page_content for doc in docs])

    # Generate response
    response_prompt = ChatPromptTemplate.from_template(
        "You are an AI assistant tasked with answering questions based on the provided context. "
        "The retrieval strategy was adapted based on the query's complexity and type. "
        "Please provide a comprehensive answer to the question, using the context when relevant "
        "and your general knowledge when necessary.\n\n"
        "Query complexity: {complexity}\n"
        "Query type: {query_type}\n"
        "Retrieval strategy: {retrieval_strategy}\n"
        "Context:\n{context}\n\n"
        "Question: {query}\n"
        "Answer:"
    )
    response_chain = response_prompt | llm
    try:
        response = response_chain.invoke({
            "complexity": complexity,
            "query_type": query_type,
            "retrieval_strategy": retrieval_strategy,
            "context": context,
            "query": query
        })
        final_answer = response.content
    except Exception as e:
        print(f"Error generating response: {e}")
        final_answer = "I apologize, but I encountered an error while generating the response."

    return {
        "query": query,
        "complexity": complexity,
        "query_type": query_type,
        "retrieval_strategy": retrieval_strategy,
        "final_answer": final_answer,
        "retrieved_context": context
    }

# Step 4: Create chunk of web data to Chroma Vector Store

In [ ]:
# Initialize the gemini-pro language model with specified settings (Change temeprature  and other parameters as per your requirement)
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.3, safety_settings={
          HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
        },)

# Load and process data
url = "https://en.wikipedia.org/wiki/Artificial_intelligence"
chunks = load_and_process_data(url)

# Create vector store
vectorstore = create_vector_store(chunks)

modules.json:   0%|          | 0.00/255 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/140 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/71.2k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

# Step 5: Run Self-Adaptive RAG

This implementation shows the key parts of Self-Adaptive RAG:

1. Dynamic assessment of query complexity
2. Adaptation of retrieval strategy based on query complexity
3. Use of different techniques (standard retrieval, query expansion, HyDE) depending on the query

In [ ]:
# Example queries
queries = [
      "What is AI?",
      "How does machine learning contribute to AI development?",
      "Discuss the ethical implications of AI in autonomous weapon systems and their potential impact on international relations."
]

# Run Self-Adaptive RAG for each query
for query in queries:
  print(f"\nQuery: {query}")
  result = self_adaptive_rag(query, vectorstore, llm)
  print(f"Complexity: {result['complexity']}")
  print(f"Query Type: {result['query_type']}")
  print(f"Retrieval Strategy: {result['retrieval_strategy']}")
  print("Final Answer:")
  print(result["final_answer"])
  print("\nRetrieved Context (first 300 characters):")
  print(result["retrieved_context"][:300] + "...")


Query: What is AI?
Complexity: 1
Query Type: Factual
Retrieval Strategy: Standard
Final Answer:
Artificial intelligence (AI) is intelligence exhibited by machines, particularly computer systems. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals.

Retrieved Context (first 300 characters):
Glossary
Glossary
vte
Artificial intelligence (AI), in its broadest sense, is intelligence exhibited by machines, particularly computer systems. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use lea...

Query: How does machine learning contribute to AI development?
Complexity: 3
Query Type: Analytical
Retrieval Strategy: Query expansion
Final Answer:
Machine learning is a crucial aspect of AI development a